In [2]:
from typing import Any
import warnings

from rl_qoc.qua.quam_components.gate_macros import CZMacro

from qm import QuantumMachinesManager, SimulationConfig
from qm.qua import *
from qm_saas import QoPVersion, QmSaas
from rl_qoc.qua.quam_components import QuAM, MeasureMacro, ResetMacro, VirtualZMacro
from rl_qoc.qua.parameter_table import ParameterTable, Parameter as QuaParameter
from rl_qoc.qua.qm_backend import FluxTunableTransmonBackend
from quam.components.macro import PulseMacro
from quam.core.operation.operations_registry import OperationsRegistry
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector, QuantumRegister
import qiskit.pulse as qp
import matplotlib.pyplot as plt
from qm import generate_qua_script
import numpy as np

import warnings
import logging

# Silence DeprecationWarnings from qiskit.pulse
warnings.filterwarnings("ignore", category=DeprecationWarning, module=r"qiskit\.pulse")
# Filter all DeprecationWarnings
logging.getLogger("qiskit").setLevel(logging.ERROR)

machine = QuAM.load(
    f"/Users/arthurostrauss/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Coding_projects/Quantum_Optimal_Control/pulse_level/qua/quam_state"
)
backend = FluxTunableTransmonBackend(machine)
print(backend.target)

Target: Transmon based QuAM
Number of qubits: 5
Instructions:
	x
		(0,)
		(1,)
		(2,)
		(3,)
		(4,)
	rz
		(0,)
		(1,)
		(2,)
		(3,)
		(4,)
	sx
		(0,)
		(1,)
		(2,)
		(3,)
		(4,)
	measure
		(0,)
		(1,)
		(2,)
		(3,)
		(4,)
	reset
		(0,)
		(1,)
		(2,)
		(3,)
		(4,)
	cz
		(0, 1)
		(1, 2)
		(2, 3)
		(3, 4)
	switch_case
	for_loop
	if_else
	while_loop



In [4]:
from rl_qoc.rewards.real_time import get_real_time_reward_circuit
from rl_qoc import (
    QEnvConfig,
    QiskitConfig,
    ExecutionConfig,
    BenchmarkConfig,
    GateTargetConfig,
)
from rl_qoc.qua import QMConfig, QMEnvironment, ParameterPool

from gymnasium.spaces import Box


def apply_parameters(qc: QuantumCircuit, params: ParameterVector, qreg: QuantumRegister):
    idx_0 = qreg.index(qreg[0])

    if len(qreg) > 1:
        idx_1 = qreg.index(qreg[1])
        with qp.build(backend) as sched:
            qp.play(qp.Constant(duration=120, amp=0.1), backend.flux_channel(idx_0))
            qp.play(
                qp.Constant(duration=120, amp=params[0]),
                backend.control_channel((idx_0, idx_1))[0],
            )
        qc.add_calibration(
            "rzz",
            [0, 1],
            sched,
            params.params if isinstance(params, ParameterVector) else params,
        )
        qc.rzz(params[0], qreg[0], qreg[1])
    else:
        with qp.build(backend) as sched:
            qp.play(
                qp.Drag(duration=120, amp=params[0], sigma=40, beta=0.2),
                backend.drive_channel(idx_0),
            )
        qc.add_calibration(
            "rx",
            [
                0,
            ],
            sched,
            params.params if isinstance(params, ParameterVector) else params,
        )
        qc.rx(params[0], qreg[0])
    with qp.build(backend, name="x_custom") as sched:
        qp.play(
            qp.Drag(duration=120, amp=0.3, sigma=40, beta=0.2),
            backend.drive_channel(idx_0),
        )
    qc.add_calibration(
        "x",
        [
            0,
        ],
        sched,
    )
    qc.add_calibration(
        "x",
        [
            1,
        ],
        sched,
    )


physical_qubits = [0, 1]
gate = "cz" if len(physical_qubits) > 1 else "x"
n_reps = [2, 5]
config = QEnvConfig(
    target=GateTargetConfig(physical_qubits, gate),
    backend_config=QiskitConfig(
        backend=backend,
        parametrized_circuit=apply_parameters,
    ),
    execution_config=ExecutionConfig(
        batch_size=1,
        n_shots=100,
        n_reps=n_reps,
        control_flow_enabled=True,
    ),
    reward="channel",
    action_space=Box(low=-1, high=1, shape=(1,), dtype=np.float32),
)

qm_config_args = {
    "input_type": "INPUT_STREAM",
    "num_updates": 500,
    "init_macro": backend.machine.apply_all_flux_to_joint_idle,
}
config.backend_config = QMConfig(**config.backend_config.as_dict(), **qm_config_args)

q_env = QMEnvironment(config)

/var/folders/5w/ygx15jq94z5dzxwkmbbj8zfm0000gn/T/ipykernel_41957/4223518635.py:21: DeprecationWarning: The function ``qiskit.pulse.builder.build()`` is deprecated as of Qiskit 1.3. It will be removed in Qiskit 2.0. The entire Qiskit Pulse package is being deprecated and will be moved to the Qiskit Dynamics repository: https://github.com/qiskit-community/qiskit-dynamics
  with qp.build(backend) as sched:
/var/folders/5w/ygx15jq94z5dzxwkmbbj8zfm0000gn/T/ipykernel_41957/4223518635.py:22: DeprecationWarning: The function ``qiskit.pulse.builder.play()`` is deprecated as of Qiskit 1.3. It will be removed in Qiskit 2.0. The entire Qiskit Pulse package is being deprecated and will be moved to the Qiskit Dynamics repository: https://github.com/qiskit-community/qiskit-dynamics
  qp.play(qp.Constant(duration=120, amp=0.1), backend.flux_channel(idx_0))
/var/folders/5w/ygx15jq94z5dzxwkmbbj8zfm0000gn/T/ipykernel_41957/4223518635.py:23: DeprecationWarning: The function ``qiskit.pulse.builder.play()``